<a href="https://colab.research.google.com/github/somoon0422/Paper/blob/main/alphapose_lstm_re0911.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I modified the Pillow installation part of the AlphaPose Colab example program.**

**This is not thoth000's original program.**


In [3]:
! pip install pyyaml==5.2
! pip install scipy
! pip install numpy
! pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113

import torch
print(torch.__version__)
import yaml, scipy, os
print(yaml.__version__)
print(scipy.__version__)

  Using cached PyYAML-5.2.tar.gz (265 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
2.4.0+cu121
6.0.2
1.13.1


In [4]:
!rm -rf /content/AlphaPose
os.chdir('/content/')
!git clone https://github.com/MVIG-SJTU/AlphaPose.git

Cloning into 'AlphaPose'...
remote: Enumerating objects: 2749, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 2749 (delta 4), reused 1 (delta 0), pack-reused 2739 (from 1)
Receiving objects: 100% (2749/2749), 118.82 MiB | 29.40 MiB/s, done.
Resolving deltas: 100% (1379/1379), done.


In [5]:
!python -m pip install cython
!sudo apt-get install libyaml-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  libyaml-doc
The following NEW packages will be installed:
  libyaml-dev
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 62.8 kB of archives.
After this operation, 257 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libyaml-dev amd64 0.2.2-1build2 [62.8 kB]
Fetched 62.8 kB in 0s (173 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libyaml-dev:amd64.
(Read

In [6]:

os.chdir('/content/AlphaPose')
print(os.getcwd())
! python setup.py build develop

/content/AlphaPose
Compiling detector/nms/src/soft_nms_cpu.pyx because it changed.
[1/1] Cythonizing detector/nms/src/soft_nms_cpu.pyx
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:85: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/trackers
copying trackers/tracker_cfg.py -> build/lib.linux-x86_64-cpython-310/trackers
copying trackers/__init__.py -> build/lib.linux-x86_64-cpython-310/trackers
copying trackers/tracker_api.py -> build/lib.linux-x86

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

! mkdir /content/AlphaPose/detector/yolo/data
file_id = '1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/detector/yolo/data/yolov3-spp.weights')

! mkdir /content/AlphaPose/detector/tracker/data
file_id = '1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/detector/tracker/data/JDE-1088x608-uncertainty')

file_id = '1kQhnMRURFiy7NsdS8EFL-8vtqEXOgECn'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/pretrained_models/fast_res50_256x192.pth')

!wget -P ./detector/yolox/data/ https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.0/yolox_x.pth



In [ ]:
import os
os.chdir('/content/AlphaPose')
! ls
! python3 scripts/demo_inference.py --cfg configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml --checkpoint pretrained_models/fast_res50_256x192.pth --indir examples/demo/ --save_img
# result json and rendered images are saved here:
! ls examples/res/
! ls examples/res/vis

In [ ]:
!pip install ultralytics

In [ ]:
import os
import cv2
import torch
import pandas as pd
import numpy as np
from ultralytics import YOLO
from alphapose.utils.config import update_config
from alphapose.models import builder
from alphapose.utils.transforms import get_func_heatmap_to_coord

class Opt:
    def __init__(self):
        self.dataset = 'coco'
        self.sp = False
        self.save_img = False
        self.outputpath = './results/'
        self.pose_flow = False
        self.vis = True

class GetKeypoint:
    NOSE = 0
    LEFT_EYE = 1
    RIGHT_EYE = 2
    LEFT_EAR = 3
    RIGHT_EAR = 4
    LEFT_SHOULDER = 5
    RIGHT_SHOULDER = 6
    LEFT_ELBOW = 7
    RIGHT_ELBOW = 8
    LEFT_WRIST = 9
    RIGHT_WRIST = 10
    LEFT_HIP = 11
    RIGHT_HIP = 12
    LEFT_KNEE = 13
    RIGHT_KNEE = 14
    LEFT_ANKLE = 15
    RIGHT_ANKLE = 16

get_keypoint = GetKeypoint()

def classify_pose(keypoints):
    # 주요 신체 부위의 y좌표 설정
    nose_y = keypoints[GetKeypoint.NOSE * 2 + 1]
    left_shoulder_y = keypoints[GetKeypoint.LEFT_SHOULDER * 2 + 1]
    right_shoulder_y = keypoints[GetKeypoint.RIGHT_SHOULDER * 2 + 1]
    left_hip_y = keypoints[GetKeypoint.LEFT_HIP * 2 + 1]
    right_hip_y = keypoints[GetKeypoint.RIGHT_HIP * 2 + 1]
    left_knee_y = keypoints[GetKeypoint.LEFT_KNEE * 2 + 1]
    right_knee_y = keypoints[GetKeypoint.RIGHT_KNEE * 2 + 1]
    left_elbow_y = keypoints[GetKeypoint.LEFT_ELBOW * 2 + 1]
    right_elbow_y = keypoints[GetKeypoint.RIGHT_ELBOW * 2 + 1]
    left_wrist_y = keypoints[GetKeypoint.LEFT_WRIST * 2 + 1]
    right_wrist_y = keypoints[GetKeypoint.RIGHT_WRIST * 2 + 1]
    left_hand_y = keypoints[GetKeypoint.LEFT_WRIST * 2 + 1]
    right_hand_y = keypoints[GetKeypoint.RIGHT_WRIST * 2 + 1]
    left_ankle_y = keypoints[GetKeypoint.LEFT_ANKLE * 2 + 1]
    right_ankle_y = keypoints[GetKeypoint.RIGHT_ANKLE * 2 + 1]

    # 주요 신체 부위의 x좌표 설정
    left_shoulder_x = keypoints[GetKeypoint.LEFT_SHOULDER * 2]
    right_shoulder_x = keypoints[GetKeypoint.RIGHT_SHOULDER * 2]
    left_hip_x = keypoints[GetKeypoint.LEFT_HIP * 2]
    right_hip_x = keypoints[GetKeypoint.RIGHT_HIP * 2]
    left_knee_x = keypoints[GetKeypoint.LEFT_KNEE * 2]
    right_knee_x = keypoints[GetKeypoint.RIGHT_KNEE * 2]
    left_elbow_x = keypoints[GetKeypoint.LEFT_ELBOW * 2]
    right_elbow_x = keypoints[GetKeypoint.RIGHT_ELBOW * 2]
    left_wrist_x = keypoints[GetKeypoint.LEFT_WRIST * 2]
    right_wrist_x = keypoints[GetKeypoint.RIGHT_WRIST * 2]
    left_hand_x = keypoints[GetKeypoint.LEFT_WRIST * 2]
    right_hand_x = keypoints[GetKeypoint.RIGHT_WRIST * 2]
    left_ankle_x = keypoints[GetKeypoint.LEFT_ANKLE * 2]
    right_ankle_x = keypoints[GetKeypoint.RIGHT_ANKLE * 2]

    # 주요 신체 부위 간의 차이 계산
    shoulder_diff = abs(left_shoulder_y - right_shoulder_y)
    hip_diff = abs(left_hip_y - right_hip_y)
    shoulder_hip_diff = min(abs(left_shoulder_y - left_hip_y), abs(right_shoulder_y - right_hip_y))
    hip_knee_diff = min(abs(left_hip_y - left_knee_y), abs(right_hip_y - right_knee_y))
    elbow_wrist_diff = min(abs(left_elbow_y - left_wrist_y), abs(right_elbow_y - right_wrist_y))
    knee_ankle_diff = min(abs(left_knee_y - left_ankle_y), abs(right_knee_y - right_ankle_y))
    elbow_diff = abs(left_elbow_y - right_elbow_y)
    foot_diff = abs(left_ankle_x - right_ankle_x)
    foot_y = min(left_ankle_y, right_ankle_y)


    if shoulder_diff < 20 and hip_diff < 20 and nose_y < min(left_shoulder_y, right_shoulder_y):
        return 'falling'
# 어깨와 엉덩이 차이가 크고,
    # 어깨와 엉덩이 차이가 크고, 힙-무릎 간 차이가 적은 경우 (공격 자세)
    elif shoulder_diff < 20 and hip_diff < 20 and shoulder_hip_diff > 50 and hip_knee_diff < 30:
        return 'attack'
    # 머리가 어깨보다 낮고, 어깨-엉덩이 차이가 큰 경우 (공격 또는 무방비 상태)
    elif nose_y < min(left_shoulder_y, right_shoulder_y) and shoulder_hip_diff > 50:
        return 'defense'
    # 어깨-엉덩이 차이가 크고, 엉덩이-무릎 간 차이가 작으며, 무릎-발목 간 차이가 큰 경우 (공격적인 자세)
    elif shoulder_hip_diff > 50 and hip_knee_diff < 30 and knee_ankle_diff > 50:
        return 'aggressive'
    # 팔꿈치-손목 간 차이가 적고, 손이 얼굴 근처에 있는 경우 (펀치 자세)
    elif elbow_wrist_diff < 20 and min(left_hand_y, right_hand_y) < nose_y + 20:
        return 'punch'
    # 팔이 앞으로 뻗어 있고 손이 몸의 앞쪽에 있는 경우 (미는 동작)
    elif elbow_wrist_diff < 20 and max(left_hand_x, right_hand_x) > max(left_shoulder_x, right_shoulder_x):
        return 'push'
    # 양손이 가까이 있고, 손이 상대방의 목 부근에 위치하는 경우 (멱살 잡는 동작)
    elif abs(left_hand_x - right_hand_x) < 20 and min(left_hand_y, right_hand_y) < nose_y + 20 and min(left_hand_y, right_hand_y) > nose_y - 20:
        return 'grab_collar'
    # 발로 차는 동작 (Kicking)
    elif knee_ankle_diff > 50 and foot_y < hip_diff:
        return 'kicking'
    # 팔꿈치로 가격하는 동작 (Elbow Strike)
    elif elbow_diff < 20 and min(left_elbow_y, right_elbow_y) < shoulder_diff:
        return 'elbow_strike'
    # 밀어 넘어뜨리는 동작 (Shoving)
    elif shoulder_hip_diff > 50 and elbow_wrist_diff < 20 and max(left_hand_x, right_hand_x) > max(left_shoulder_x, right_shoulder_x):
        return 'shoving'
    # 목을 조르는 동작 (Choking)
    elif abs(left_hand_x - right_hand_x) < 20 and min(left_hand_y, right_hand_y) < nose_y + 10 and min(left_hand_y, right_hand_y) > nose_y - 10:
        return 'choking'
    # 발로 밟는 동작 (Stomping)
    elif foot_diff > 50 and foot_y > min(left_ankle_y, right_ankle_y):
        return 'stomping'
    # 웅크리고 있는 자세 (Crouching)
    elif shoulder_hip_diff < 50 and hip_knee_diff < 50 and min(left_knee_y, right_knee_y) < nose_y:
        return 'crouching'
    else:
        return 'neutral'

def vis_frame(orig_img, boxes, keypoints, labels):
    """
    이미지를 시각화하기 위한 함수
    :param orig_img: 원본 이미지
    :param boxes: 바운딩 박스 리스트
    :param keypoints: 키포인트 리스트
    :param labels: 포즈 라벨 리스트
    :return: 시각화된 이미지
    """
    vis_img = orig_img.copy()
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        for j in range(0, len(keypoints[i]), 2):
            cv2.circle(vis_img, (int(keypoints[i][j]), int(keypoints[i][j + 1])), 5, (0, 255, 0), -1)
        cv2.putText(vis_img, labels[i], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return vis_img

def main():
    # YOLOv8 모델 로드
    yolo_model = YOLO('yolov8n.pt')

    image_files = []

    image_directory = '/content/drive/MyDrive/논문주제/path_to_output_frames/train/Fight'
    for root, dirs, files in os.walk(image_directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))

    image_files = sorted(image_files)

    output_directory = '/content/drive/MyDrive/논문주제/results4/'
    csv_file_path = '/content/drive/MyDrive/논문주제/Final_project/Fight_pose_re_falling_keypoint.csv'
    pretrained_model_path = '/content/drive/MyDrive/논문주제/Final_project/pretrained_models/fast_res50_256x192.pth'

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # 설정 파일 경로
    cfg_file = '/content/AlphaPose/configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml'

    # 설정 업데이트
    cfg = update_config(cfg_file)
    cfg['checkpoint'] = pretrained_model_path
    cfg['outputpath'] = output_directory
    cfg['vis'] = True

    # 결과 저장 디렉토리 생성
    os.makedirs(cfg['outputpath'], exist_ok=True)

    # GPU 장치 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # AlphaPose 모델 로드
    pose_model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)
    print(f'Loading pose model from {cfg["checkpoint"]}')
    pose_model.load_state_dict(torch.load(cfg["checkpoint"], map_location=device))
    pose_model = torch.nn.DataParallel(pose_model).to(device)
    pose_model.eval()

    # 컬럼 이름 설정 (두 명의 사람에 대한 데이터 포함)
        # 컬럼 이름 설정 (두 명의 사람에 대한 데이터 포함)
    header = [
    'image_name', 'target_person_label',
    'target_person_nose_x', 'target_person_nose_y', 'target_person_left_eye_x', 'target_person_left_eye_y',
    'target_person_right_eye_x', 'target_person_right_eye_y', 'target_person_left_ear_x', 'target_person_left_ear_y',
    'target_person_right_ear_x', 'target_person_right_ear_y', 'target_person_left_shoulder_x', 'target_person_left_shoulder_y',
    'target_person_right_shoulder_x', 'target_person_right_shoulder_y', 'target_person_left_elbow_x', 'target_person_left_elbow_y',
    'target_person_right_elbow_x', 'target_person_right_elbow_y', 'target_person_left_wrist_x', 'target_person_left_wrist_y',
    'target_person_right_wrist_x', 'target_person_right_wrist_y', 'target_person_left_hip_x', 'target_person_left_hip_y',
    'target_person_right_hip_x', 'target_person_right_hip_y', 'target_person_left_knee_x', 'target_person_left_knee_y',
    'target_person_right_knee_x', 'target_person_right_knee_y', 'target_person_left_ankle_x', 'target_person_left_ankle_y',
    'target_person_right_ankle_x', 'target_person_right_ankle_y', 'closest_person_label',
    'closest_person_nose_x', 'closest_person_nose_y', 'closest_person_left_eye_x', 'closest_person_left_eye_y',
    'closest_person_right_eye_x', 'closest_person_right_eye_y', 'closest_person_left_ear_x', 'closest_person_left_ear_y',
    'closest_person_right_ear_x', 'closest_person_right_ear_y', 'closest_person_left_shoulder_x', 'closest_person_left_shoulder_y',
    'closest_person_right_shoulder_x', 'closest_person_right_shoulder_y', 'closest_person_left_elbow_x', 'closest_person_left_elbow_y',
    'closest_person_right_elbow_x', 'closest_person_right_elbow_y', 'closest_person_left_wrist_x', 'closest_person_left_wrist_y',
    'closest_person_right_wrist_x', 'closest_person_right_wrist_y', 'closest_person_left_hip_x', 'closest_person_left_hip_y',
    'closest_person_right_hip_x', 'closest_person_right_hip_y', 'closest_person_left_knee_x', 'closest_person_left_knee_y',
    'closest_person_right_knee_x', 'closest_person_right_knee_y', 'closest_person_left_ankle_x', 'closest_person_left_ankle_y',
    'closest_person_right_ankle_x', 'closest_person_right_ankle_y'
]
    all_keypoints = []

    # 이전에 저장된 체크포인트 확인
    if os.path.exists(csv_file_path):
        df_existing = pd.read_csv(csv_file_path)
        processed_images = set(df_existing['image_name'].tolist())
    else:
        df_existing = pd.DataFrame(columns=header)
        processed_images = set()

    # 모든 이미지 파일 처리
    for idx, image_file in enumerate(image_files):
        if image_file in processed_images:
            continue  # 이미 처리된 이미지라면 건너뛰기

        input_image_path = os.path.join(image_directory, image_file)
        input_image = cv2.imread(input_image_path)
        if input_image is None:
            print(f'Error: Image {input_image_path} not found or could not be read.')
            continue

        # YOLO 모델 실행 후 사람 객체 필터링
        results = yolo_model.predict(input_image, save=False, classes=[0])
        human_detections = [d for d in results[0].boxes.data.cpu().numpy() if int(d[-1]) == 0]

        if not human_detections:
            print(f'No human detections in image {input_image_path}')
            continue

        inps = []
        boxes = []
        for detection in human_detections:
            x1, y1, x2, y2 = map(int, detection[:4])
            boxes.append([x1, y1, x2, y2])
            inp = cv2.resize(input_image[y1:y2, x1:x2], (cfg.DATA_PRESET.IMAGE_SIZE[0], cfg.DATA_PRESET.IMAGE_SIZE[1]))
            inps.append(inp)

        inps = torch.stack([torch.from_numpy(np.array(inp)).permute(2, 0, 1).float() for inp in inps]).to(device)

        # AlphaPose 모델로 스켈레톤 추출
        with torch.no_grad():
            hm = pose_model(inps)

        # 스켈레톤 결과를 처리하여 CSV 파일로 저장
        keypoints = []
        labels = []
        for i, box in enumerate(boxes):
            preds, maxvals = get_func_heatmap_to_coord(cfg)(hm[i], box)
            keypoints_flatten = preds.flatten().tolist()
            keypoints.append(keypoints_flatten)
            labels.append(classify_pose(keypoints_flatten))

        # 100번에 한 번씩 이미지를 저장
        if idx % 100 == 0:
            result_img = vis_frame(input_image, boxes, keypoints, labels)
            output_path = os.path.join(output_directory, f'result_{os.path.basename(input_image_path)}')
            cv2.imwrite(output_path, result_img)
            print(f'Results saved at {output_path}')


        # 두 명의 사람에 대한 키포인트 저장 및 조건 처리
        if len(keypoints) >= 2:
            # 넘어진 사람이 있는지 확인
            falling_indices = [i for i, label in enumerate(labels) if label == 'falling']

            if falling_indices:
                # 넘어진 사람의 인덱스와 키포인트
                fallen_index = falling_indices[0]
                fallen_keypoints = keypoints[fallen_index]

                # 넘어진 사람과 가장 가까운 사람 찾기
                min_distance = float('inf')
                closest_person_index = None

                for i, kp in enumerate(keypoints):
                    if i != fallen_index:
                        # 두 사람 간의 유클리드 거리 계산 (코어 위치 기준: 엉덩이 좌표 사용)
                        distance = np.linalg.norm(np.array(fallen_keypoints[GetKeypoint.LEFT_HIP*2:GetKeypoint.LEFT_HIP*2+2]) -
                                                  np.array(kp[GetKeypoint.LEFT_HIP*2:GetKeypoint.LEFT_HIP*2+2]))
                        if distance < min_distance:
                            min_distance = distance
                            closest_person_index = i

                # 넘어진 사람과 가장 가까운 사람의 키포인트 저장
                keypoints_data = [image_file, labels[fallen_index]] + keypoints[fallen_index] + [labels[closest_person_index]] + keypoints[closest_person_index]

            else:
                # 웅크리고 있는 사람이 있는지 확인
                crouching_indices = [i for i, label in enumerate(labels) if label == 'crouching']

                if crouching_indices:
                    # 웅크린 사람의 인덱스와 키포인트
                    crouching_index = crouching_indices[0]
                    crouching_keypoints = keypoints[crouching_index]

                    # 웅크린 사람과 가장 가까운 사람 찾기
                    min_distance = float('inf')
                    closest_person_index = None

                    for i, kp in enumerate(keypoints):
                        if i != crouching_index:
                            # 두 사람 간의 유클리드 거리 계산 (코어 위치 기준: 엉덩이 좌표 사용)
                            distance = np.linalg.norm(np.array(crouching_keypoints[GetKeypoint.LEFT_HIP*2:GetKeypoint.LEFT_HIP*2+2]) -
                                                      np.array(kp[GetKeypoint.LEFT_HIP*2:GetKeypoint.LEFT_HIP*2+2]))
                            if distance < min_distance:
                                min_distance = distance
                                closest_person_index = i

                    # 웅크린 사람과 가장 가까운 사람의 키포인트 저장
                    keypoints_data = [image_file, labels[crouching_index]] + keypoints[crouching_index] + [labels[closest_person_index]] + keypoints[closest_person_index]

                else:
                    # 넘어진 사람이나 웅크린 사람이 없을 경우, 가장 가까운 두 사람을 선택
                    min_distance = float('inf')
                    person1_index, person2_index = None, None

                    for i in range(len(keypoints)):
                        for j in range(i + 1, len(keypoints)):
                            # 두 사람 간의 유클리드 거리 계산 (코어 위치 기준: 엉덩이 좌표 사용)
                            distance = np.linalg.norm(np.array(keypoints[i][GetKeypoint.LEFT_HIP*2:GetKeypoint.LEFT_HIP*2+2]) -
                                                      np.array(keypoints[j][GetKeypoint.LEFT_HIP*2:GetKeypoint.LEFT_HIP*2+2]))
                            if distance < min_distance:
                                min_distance = distance
                                person1_index, person2_index = i, j

                    # 가장 가까운 두 사람의 키포인트 저장
                    keypoints_data = [image_file, labels[person1_index]] + keypoints[person1_index] + [labels[person2_index]] + keypoints[person2_index]

        elif len(keypoints) == 1:
            # 한 명만 있는 경우, 그 사람의 데이터와 기본값 저장
            keypoints_data = [image_file, labels[0]] + keypoints[0] + [''] + [0] * 34

        else:
            continue

        # 주기적으로 CSV 파일 저장
        if idx % 10 == 0:  # 매번 이미지마다 CSV 파일에 저장
            df_temp = pd.DataFrame(all_keypoints, columns=header)
            df_temp.to_csv(csv_file_path, mode='a', header=not os.path.exists(csv_file_path), index=False)
            print(f'Progress saved at {csv_file_path}')
            all_keypoints = []  # 임시 리스트 초기화

        # 여기서 keypoints_data를 all_keypoints 리스트에 추가
        all_keypoints.append(keypoints_data)


    if all_keypoints:
        df_temp = pd.DataFrame(all_keypoints, columns=header)
        # 동일한 이미지 이름의 데이터가 이미 존재하는지 확인
        df_existing = pd.read_csv(csv_file_path)
        df_combined = pd.concat([df_existing, df_temp], ignore_index=True)

        # 중복된 이미지 이름 제거 (최근의 데이터만 유지)
        df_combined.drop_duplicates(subset=['image_name'], keep='last', inplace=True)

        df_combined.to_csv(csv_file_path, index=False)
        print(f'Progress saved at {csv_file_path}')
        all_keypoints = []  # 임시 리스트 초기화



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
if __name__ == '__main__':
  main()

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/논문주제/Final_project/train_pose2_keypoint.csv')

df.head(10)

In [ ]:
row_lengths = df.apply(len, axis=1)

# 첫 5개의 행의 길이 출력
print("첫 5개 행의 길이:", row_lengths.head())

# 전체 데이터에 대한 통계(최소, 최대, 평균 길이) 확인
print("최소 길이:", row_lengths.min())
print("최대 길이:", row_lengths.max())
print("평균 길이:", row_lengths.mean())

# 전체 데이터 길이 확인
print("전체 데이터 길이:", len(df))

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/논문주제/Final_project/NonFight_pose2_keypoint.csv')

df2.head(10)

In [ ]:
df2.columns

In [ ]:
row_lengths2 = df2.apply(len, axis=1)

# 첫 5개의 행의 길이 출력
print("첫 5개 행의 길이:", row_lengths2.head())

# 전체 데이터에 대한 통계(최소, 최대, 평균 길이) 확인
print("최소 길이:", row_lengths2.min())
print("최대 길이:", row_lengths2.max())
print("평균 길이:", row_lengths2.mean())

# 전체 데이터 길이 확인
print("전체 데이터 길이:", len(df2))

In [ ]:
df2['closest_person_label'] = df2['closest_person_label'].apply(lambda x: 'not' + x if isinstance(x, str) else x)

In [ ]:
df2

In [ ]:
df2.to_csv('/content/drive/MyDrive/논문주제/Final_project/NonFight_pose2_keypoint.csv', index=False)

In [ ]:
import pandas as pd

# 각 파일 경로를 리스트로 저장
file_paths = [
    '/content/drive/MyDrive/논문주제/Final_project/train_pose2_keypoint.csv',
    '/content/drive/MyDrive/논문주제/Final_project/NonFight_pose2_keypoint.csv'
]

# 빈 데이터프레임 생성
combined_df = pd.DataFrame()

# 각 파일을 불러와서 합치기
for file_path in file_paths:
    # CSV 파일 불러오기
    df = pd.read_csv(file_path)

    # 모든 값이 채워진 행 선택
    complete_rows = df.dropna()

    # 30,000개의 행을 선택 (부족할 경우 결측치가 있는 행 추가)
    if len(complete_rows) >= 30000:
        selected_df = complete_rows.iloc[:30000]
    else:
        # 결측치가 있는 행 선택
        incomplete_rows = df[df.isna().any(axis=1)]
        # 부족한 부분을 결측치가 있는 행으로 채움
        remaining_rows = 30000 - len(complete_rows)
        selected_df = pd.concat([complete_rows, incomplete_rows.iloc[:remaining_rows]], ignore_index=True)

    # 결과를 combined_df에 추가
    combined_df = pd.concat([combined_df, selected_df], ignore_index=True)

# 결과 출력
print(combined_df.shape)

In [ ]:
# 얼굴 관련 키포인트 제거
columns_to_remove = [
    'target_person_nose_x', 'target_person_nose_y', 'target_person_left_eye_x', 'target_person_left_eye_y',
    'target_person_right_eye_x', 'target_person_right_eye_y', 'target_person_left_ear_x', 'target_person_left_ear_y',
    'target_person_right_ear_x', 'target_person_right_ear_y',
    'closest_person_nose_x', 'closest_person_nose_y', 'closest_person_left_eye_x', 'closest_person_left_eye_y',
    'closest_person_right_eye_x', 'closest_person_right_eye_y', 'closest_person_left_ear_x', 'closest_person_left_ear_y',
    'closest_person_right_ear_x', 'closest_person_right_ear_y'
]


# 데이터프레임에서 해당 컬럼 제거
combined_df = combined_df.drop(columns=columns_to_remove)

# 결과 확인
print(combined_df.head())

In [ ]:
import numpy as np
import pandas as pd

# 얼굴 관련 키포인트 제거 후 남은 keypoints 리스트 정의
keypoints = [
    'target_person_left_shoulder_x', 'target_person_left_shoulder_y',
    'target_person_right_shoulder_x', 'target_person_right_shoulder_y',
    'target_person_left_elbow_x', 'target_person_left_elbow_y',
    'target_person_right_elbow_x', 'target_person_right_elbow_y',
    'target_person_left_wrist_x', 'target_person_left_wrist_y',
    'target_person_right_wrist_x', 'target_person_right_wrist_y',
    'target_person_left_hip_x', 'target_person_left_hip_y',
    'target_person_right_hip_x', 'target_person_right_hip_y',
    'target_person_left_knee_x', 'target_person_left_knee_y',
    'target_person_right_knee_x', 'target_person_right_knee_y',
    'target_person_left_ankle_x', 'target_person_left_ankle_y',
    'target_person_right_ankle_x', 'target_person_right_ankle_y',
    'closest_person_left_shoulder_x', 'closest_person_left_shoulder_y',
    'closest_person_right_shoulder_x', 'closest_person_right_shoulder_y',
    'closest_person_left_elbow_x', 'closest_person_left_elbow_y',
    'closest_person_right_elbow_x', 'closest_person_right_elbow_y',
    'closest_person_left_wrist_x', 'closest_person_left_wrist_y',
    'closest_person_right_wrist_x', 'closest_person_right_wrist_y',
    'closest_person_left_hip_x', 'closest_person_left_hip_y',
    'closest_person_right_hip_x', 'closest_person_right_hip_y',
    'closest_person_left_knee_x', 'closest_person_left_knee_y',
    'closest_person_right_knee_x', 'closest_person_right_knee_y',
    'closest_person_left_ankle_x', 'closest_person_left_ankle_y',
    'closest_person_right_ankle_x', 'closest_person_right_ankle_y'
]

# 시퀀스 길이 설정
n_timesteps = 20

# 레이블을 원-핫 인코딩하는 함수
def one_hot_encode(labels):
    unique_labels = np.unique(labels)
    label_map = {label: idx for idx, label in enumerate(unique_labels)}
    encoded = np.array([label_map[label] for label in labels])
    return np.eye(len(unique_labels))[encoded]

# 시퀀스 데이터를 생성하는 함수
def create_sequences(data, seq_length):
    X = []
    y = []

    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i + seq_length]
        label = data.iloc[i + seq_length - 1]['target_person_label']

        # 필요 없는 컬럼을 제외하고 시퀀스 저장
        X.append(seq[keypoints].values)
        y.append(label)

    return np.array(X), np.array(y)

# 30,000개의 로우만 사용하도록 데이터프레임을 슬라이스
subset_df = combined_df.iloc[:60000]

# 시퀀스 생성
X, y = create_sequences(subset_df, n_timesteps)

# 레이블 원-핫 인코딩
y = one_hot_encode(y)

# 입력 데이터와 출력 데이터의 shape 확인
print("X의 shape:", X.shape)
print("y의 shape:", y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# 학습 데이터와 테스트 데이터 분리
# train test split
# 학습 데이터와 테스트 데이터 분리 (stratify 옵션 제거)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=7461
)
X_train.shape

In [ ]:
import tensorflow as tf

def create_dataset(X, y, batch_size=32, shuffle=True):
    # 데이터셋 객체 생성
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    # 데이터셋 섞기
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(X))

    # 배치 크기로 분할
    dataset = dataset.batch(batch_size)

    return dataset

# 배치 사이즈 설정
batch_size = 64

# 데이터 세트 생성
dataset_train = create_dataset(X_train, y_train, batch_size)
dataset_test = create_dataset(X_test, y_test, batch_size)

In [ ]:
len(dataset_train)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# 모델 파라미터 설정
input_shape = (X.shape[1], X.shape[2])  # 입력 시퀀스의 형태 (시퀀스 길이, 특성 수)
num_classes = y.shape[1]  # 출력 클래스 수

# Early stopping 설정
early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

# 모델 생성 함수 정의
def build_model(lstm_layers, input_shape, num_classes):
    model = Sequential()

    # LSTM layers
    units = [128, 64, 32]  # 각 LSTM 레이어의 유닛 수
    for i in range(lstm_layers):
        return_sequences = i < lstm_layers - 1  # 마지막 LSTM layer는 return_sequences=False
        model.add(LSTM(units[i], return_sequences=return_sequences, activation='tanh', input_shape=input_shape if i == 0 else None))
        model.add(LayerNormalization())
        model.add(Dropout(0.2))

    # Dense layers
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# LSTM layer를 1, 2, 3개 사용하는 모델을 각각 생성 및 훈련
models = {}
for lstm_layers in [1, 2, 3]:
    print(f"Training model with {lstm_layers} LSTM layer(s)...")
    model = build_model(lstm_layers, input_shape, num_classes)
    model.summary()
    history = model.fit(dataset_train, epochs=100, callbacks=[early_stopping], validation_data=dataset_test, verbose=0)

    # 검증 데이터에 대한 예측 수행
    y_pred = model.predict(dataset_test)

    # 예측된 확률을 클래스 레이블로 변환
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)  # 실제 클래스 레이블

    # 정밀도, 재현율, F1 점수 계산
    precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
    recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

    # 모델 결과 저장
    models[f'{lstm_layers}_LSTM'] = {
        'accuracy': history.history['val_accuracy'][-1],
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

    print(f"Model with {lstm_layers} LSTM layer(s):")
    print(f"Accuracy: {history.history['val_accuracy'][-1]}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}\n")

In [ ]:
import matplotlib.pyplot as plt

# 모델의 성능 지표를 시각화하기 위한 함수 정의
def plot_model_performance(models):
    lstm_layers = ['1_LSTM', '2_LSTM', '3_LSTM']  # LSTM 레이어 수에 따른 키 설정
    metrics = ['accuracy', 'precision', 'recall', 'f1_score']  # 시각화할 성능 지표

    for metric in metrics:
        plt.figure(figsize=(8, 6))
        values = [models[layer][metric] for layer in lstm_layers]
        plt.plot(lstm_layers, values, marker='o', linestyle='-', label=metric)
        plt.title(f'모델 성능 - {metric.capitalize()}')
        plt.xlabel('LSTM 레이어 수')
        plt.ylabel(metric.capitalize())
        plt.grid(True)
        plt.show()

# 모델의 성능 지표를 시각화
plot_model_performance(models)

In [ ]:
# 모델을 HDF5 파일 형식으로 저장
model.save('/content/drive/MyDrive/논문주제/Final_project/not_my_model2_lstm.h5')

In [ ]:
# 훈련 과정 시각화 (선택적)
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('lstm3')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend()
plt.show()

---

영상테스트

In [ ]:
!wget -q -O /content/DejaVuSans-Bold.ttf https://github.com/dejavu-fonts/dejavu-fonts/raw/master/ttf/

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow  # 코랩 환경에서 이미지 표시를 위한 모듈
import os

# 영상 파일 위치 지정
video_file = '/content/drive/MyDrive/논문주제/Final_project/Dataset/학교폭력 영상/F_176_1_0_0_0.mp4'

# YOLO 모델 정의 (학습 당시에는 v8x로 진행)
yolo_model = YOLO('yolov8n.pt')

# LSTM 모델 정의
lstm_model = load_model('/content/drive/MyDrive/논문주제/Final_project/my_model_lstm2.h5')

# 시퀀스 정의
sequences = []

# 시퀀스 길이
sequence_length = 8

# 동작 라벨 정의
actions = [
    "falling", "attack", "defense", "aggressive", "punch", "push",
    "grab_collar", "kicking", "elbow_strike", "shoving",
    "choking", "stomping", "crouching", "neutral"
]

# 위험한 동작 라벨 정의
dangerous_actions = [
    "falling", "attack", "aggressive", "punch", "grab_collar",
    "kicking", "elbow_strike", "shoving", "choking", "stomping"
]

# 이전에 그려진 박스의 좌표를 저장할 변수
prev_box = None

# 동영상 파일로부터 비디오 캡처 객체 생성
cap = cv2.VideoCapture(video_file)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)


def extract_keypoint(keypoints):
    # keypoints: [x, y, confidence, ...]
    return keypoints[:, :2].flatten().tolist()  # x, y 좌표만 사용

def find_closest_pairs(boxes, labels, keypoints):
    """
    주어진 조건에 따라 두 사람 간의 가장 적합한 바운딩 박스를 반환합니다.
    """
    min_distance = float('inf')
    closest_pair = None

    # 쓰러진 사람이 있는지 확인
    falling_indices = [i for i, label in enumerate(labels) if label == 'falling']
    if falling_indices:
        fallen_index = falling_indices[0]
        fallen_keypoints = keypoints[fallen_index]

        for i, kp in enumerate(keypoints):
            if i != fallen_index:
                distance = np.linalg.norm(np.array(fallen_keypoints[GetKeypoint.LEFT_HIP*2:GetKeypoint.LEFT_HIP*2+2]) -
                                          np.array(kp[GetKeypoint.LEFT_HIP*2:GetKeypoint.LEFT_HIP*2+2]))
                if distance < min_distance:
                    min_distance = distance
                    closest_pair = (fallen_index, i)

    # 웅크리고 있는 사람이 있는지 확인
    elif not closest_pair:
        crouching_indices = [i for i, label in enumerate(labels) if label == 'crouching']
        if crouching_indices:
            crouching_index = crouching_indices[0]
            crouching_keypoints = keypoints[crouching_index]

            for i, kp in enumerate(keypoints):
                if i != crouching_index:
                    distance = np.linalg.norm(np.array(crouching_keypoints[GetKeypoint.LEFT_HIP*2:GetKeypoint.LEFT_HIP*2+2]) -
                                              np.array(kp[GetKeypoint.LEFT_HIP*2:GetKeypoint.LEFT_HIP*2+2]))
                    if distance < min_distance:
                        min_distance = distance
                        closest_pair = (crouching_index, i)

    # 두 사람이 있는 경우, 가장 가까운 두 사람을 선택
    if not closest_pair:
        for i in range(len(boxes)):
            for j in range(i + 1, len(boxes)):
                center_i = [(boxes[i][0] + boxes[i][2]) / 2, (boxes[i][1] + boxes[i][3]) / 2]
                center_j = [(boxes[j][0] + boxes[j][2]) / 2, (boxes[j][1] + boxes[j][3]) / 2]
                distance = np.linalg.norm(np.array(center_i) - np.array(center_j))

                if distance < min_distance:
                    min_distance = distance
                    closest_pair = (i, j)

    return [boxes[closest_pair[0]], boxes[closest_pair[1]]]

def create_largest_box(box1, box2):
    """
    두 사람의 바운딩 박스를 감싸는 가장 큰 바운딩 박스를 생성합니다.
    """
    x1 = min(box1[0], box2[0])
    y1 = min(box1[1], box2[1])
    x2 = max(box1[2], box2[2])
    y2 = max(box1[3], box2[3])
    return [x1, y1, x2, y2]

def calculate_font_center(box):
    """
    바운딩 박스의 중앙 위치를 계산하여 텍스트를 표시할 위치를 반환합니다.
    """
    center_x = (box[0] + box[2]) / 2
    center_y = (box[1] + box[3]) / 2
    return center_x, center_y

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    results = yolo_model(frame, save=False, classes=[0])

    if not results:
        continue

    # 키포인트가 있는지 확인하고 처리
    if results[0].keypoints is not None:
        results_keypoint = results[0].keypoints.xyn.cpu().numpy()
        labels = [classify_pose(extract_keypoint(kp)) for kp in results_keypoint]

        if len(results_keypoint) == 1:
            keypoint_list = extract_keypoint(results_keypoint[0])
            keypoint_list.extend([0] * 24)
        elif len(results_keypoint) >= 2:
            xy_list = results[0].boxes.xyxy.cpu().numpy().tolist()
            closest_boxes = find_closest_pairs(xy_list, labels, results_keypoint)
            keypoint_list1 = extract_keypoint(results_keypoint[0])
            keypoint_list2 = extract_keypoint(results_keypoint[1])
            keypoint_list = keypoint_list1 + keypoint_list2
        else:
            continue

        sequences.append(keypoint_list)

    if len(sequences) == sequence_length:
        sequences_array = np.array(sequences).reshape(1, sequence_length, -1)
        predictions = lstm_model.predict(sequences_array)

        action_index = np.argmax(predictions)
        action_name = actions[action_index]
        max_value = np.max(predictions)

        if action_name in dangerous_actions and max_value > 0.9:
            box_color = (0, 0, 255)
            last_dangerous_action_detected = action_name
        elif max_value > 0.5:
            box_color = (0, 128, 255)
            last_dangerous_action_detected = None
        else:
            box_color = (0, 255, 0)
            last_dangerous_action_detected = None

        largest_box = create_largest_box(*closest_boxes)

        prev_box = largest_box
        sequences.clear()

    if prev_box is not None:
        pil_im = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_im)

        draw.rectangle(((prev_box[0], prev_box[1]), (prev_box[2], prev_box[3])), outline=box_color, width=5)
        draw.text((prev_box[0], prev_box[1] - 40), action_name, font=font, fill=box_color)

        if last_dangerous_action_detected:
            danger_text = f"Danger! {last_dangerous_action_detected} detected!"
            draw.text((50, 100), danger_text, font=font, fill=(255, 0, 0))

        frame = cv2.cvtColor(np.array(pil_im), cv2.COLOR_RGB2BGR)

    cv2_imshow(frame)  # 코랩 환경에서는 cv2_imshow를 사용
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()